In [1]:
import sys
import os
root_dir = '/home/qing/Desktop/Closed-Loop-Learning/HINT/'
os.chdir(root_dir)
from train import *

In [2]:
sys.argv = []
args = parse_args()
args.input = 'symbol'
args.model = 'TRAN.relative_universal'
args.resume = f'./models/{args.input}.{args.model}/model_100000.p'
args.nhead = 8
args.enc_layers = 6
args.dec_layers = 1
args.hid_dim = 512
args.max_rel_pos = 15
args

Namespace(batch_size=128, cos_sim_margin=0.2, curriculum='no', dec_layers=1, dropout=0.5, early_stop=None, emb_dim=128, enc_layers=6, epochs=10, epochs_eval=1, fewshot=None, grad_clip=5.0, hid_dim=512, input='symbol', iterations=None, iterations_eval=None, layers=1, lr=0.001, lr_scheduler='constant', main_dataset_ratio=0, max_op_train=None, max_rel_pos=15, model='TRAN.relative_universal', nhead=8, output_dir='outputs/', perception_pretrain='data/perception_pretrain/model.pth.tar_78.2_match', pos_emb_type='sin', result_encoding='decimal', resume='./models/symbol.TRAN.relative_universal/model_100000.p', save_model=False, seed=0, train_size=None, wandb='HINT', warmup_steps=100)

In [3]:
train_set = HINT('train', input=args.input, fewshot=args.fewshot, 
                    n_sample=args.train_size, max_op=args.max_op_train,
                    main_dataset_ratio=args.main_dataset_ratio)
val_set = HINT('val', input=args.input, fewshot=args.fewshot)
test_set = HINT('test', input=args.input, fewshot=args.fewshot)
print('train:', len(train_set), 'val:', len(val_set), 'test:', len(test_set))

args.train_set = train_set
args.val_set = val_set
args.test_set = test_set

train: 998000 val: 4698 test: 46620


In [4]:
args.res_enc = ResultEncoding(args.result_encoding)
model = make_model(args)
if args.resume:
    print('Load checkpoint from ' + args.resume)
    ckpt = torch.load(args.resume)
    model.load_state_dict(ckpt['model_state_dict'])
model.to(DEVICE)

n_params = sum(p.numel() for p in model.parameters())
print('Num params:', n_params)

Load checkpoint from ./models/symbol.TRAN.relative_universal/model_100000.p
Num params: 8019469


In [5]:
dataset = args.test_set
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                            shuffle=False, num_workers=4, collate_fn=HINT_collate)

model.eval() 
res_all = []
res_pred_all = []

expr_all = []
expr_pred_all = []

dep_all = []
dep_pred_all = []

metrics = OrderedDict()

with torch.no_grad():
    for sample in tqdm(dataloader):
        if args.input == 'image':
            src = sample['img_seq']
        elif args.input == 'symbol':
            src = torch.tensor([x for s in sample['sentence'] for x in s])
        res = sample['res']
        if args.result_encoding == 'sin':
            tgt = res.unsqueeze(1)
        else:
            tgt = torch.tensor(args.res_enc.res2seq_batch(res.numpy()))
        expr = sample['expr']
        dep = sample['head']
        src_len = sample['len']

        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        output = model(src, tgt[:, :-1], src_len)
        pred = torch.argmax(output, -1).detach().cpu().numpy()
        if args.result_encoding == 'sin':
            res_pred = pred
        else:
            res_pred = args.res_enc.seq2res_batch(pred)
        res_pred_all.append(res_pred)
        res_all.append(res)

        # expr_pred_all.extend(expr_preds)
        expr_all.extend(expr)
        # dep_pred_all.extend(dep_preds)
        dep_all.extend(dep)

res_pred_all = np.concatenate(res_pred_all, axis=0)
res_all = np.concatenate(res_all, axis=0)
result_acc = (res_pred_all == res_all).mean()
metrics['result_acc/avg'] = result_acc

tracked_attrs = ['length', 'symbol', 'digit', 'result', 'eval', 'tree_depth', 'ps_depth', 'max_dep']
for attr in tracked_attrs:
    # print(f"result accuracy by {attr}:")
    attr2ids = getattr(dataloader.dataset, f'{attr}2ids')
    for k, ids in sorted(attr2ids.items()):
        res = res_all[ids]
        res_pred = res_pred_all[ids]
        res_acc = (res == res_pred).mean() if ids else 0.
        k = 'div' if k == '/' else k
        metrics[f'result_acc/{attr}/{k}'] = res_acc
        # print(k, "(%2d%%)"%(100*len(ids)//len(dataloader.dataset)), "%5.2f"%(100 * res_acc))

100%|█████████████████████████████████████████| 729/729 [00:20<00:00, 35.88it/s]


In [7]:
error_ids = [i for i, (x, y) in enumerate(zip(res_all, res_pred_all)) if x != y ]

In [21]:
symbol_images_dir = root_dir + 'data/symbol_images/'
def render_img(img_paths):
    images = [Image.open(symbol_images_dir + x) for x in img_paths]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('L', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    return new_im

def show_sample(sample, show_image=True):
    print(sample['expr'], len(sample['expr']))
    if show_image:
        img = render_img(sample['img_paths'])
        display(img)

In [25]:
for i in range(len(dataset)):
    sample = dataset.dataset[i]
#     if sample['eval'] not in ['LL']: continue
    if sample['res'] < 100: continue
    print(sample['expr'], len(sample['expr']), max(sample['res_all']))
    print(res_all[i], res_pred_all[i])
    input()

8*(8*9) 7 576
576 64


6*(9+9) 7 108
108 98


6*(9+8) 7 102
102 92


9*(2*7) 7 126
126 36


8*8*2 5 128
128 64


...

4*8*8 5 256
256 96


(8+4)*9 7 108
108 72


9*(7*4) 7 252
252 72


9*(6*5) 7 270
270 90


KeyboardInterrupt: Interrupted by user